In [1]:
import os

In [4]:
%pwd

'g:\\End-to-End-ML-Project\\End-to-end-Ml-project'

In [3]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    #constants dir contains path of config, paarams, schema
    def __init__(self, config_pathfile = CONFIG_PATH_FILE, schema_pathfile = SCHEMA_PATH_FILE, 
                 params_pathfile = PARMAS_PATH_FILE):      
            
        self.config = read_yaml(config_pathfile) 
        self.params = read_yaml(params_pathfile)
        self.schema = read_yaml(schema_pathfile)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path= config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name      
            )
        return model_evaluation_config

In [7]:
import os
import joblib
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def save_result(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis = 1)
        y_test = test_data[[self.config.target_column]]

        y_pred = model.predict(X_test)

        (rmse, mae, r2) = self.eval_metric(y_test, y_pred)

        scores = {'rmse':rmse, 'mae': mae, 'r2':r2}
        save_json(path = Path(self.config.metric_file_name), data = scores)

In [ ]:
try: 
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config) 
    model_evaluation_config.save_result()
except Exception as e:
    raise e